In [1]:
import pandas as pd
import numpy as np
import Levenshtein

In [2]:
def merge_matchdays(year):
    df_final = pd.DataFrame()
    for i in range(1, 39):
        df = pd.read_excel(f"data/{year}/Voti_Fantacalcio_Stagione_{year-1}-{year-2000}_Giornata_{i}.xlsx", skiprows=5)
        # toglie righe con nomi squadra
        df = df.dropna(thresh=2)
        # toglie nomi colonne ripetuti
        df = df[df["Nome"]!="Nome"]
        # toglie allenatori
        df = df[df["Ruolo"]!= "ALL"]
        # toglie 6 politici
        df = df[df["Voto"]!= "6*"]
        df.loc[:, "Giornata"] = i
        df_final = pd.concat([df_final, df])
    return df_final

In [3]:
df_all_16 = merge_matchdays(2016)

In [4]:
df_all_16.head(5)

,Cod.,Ruolo,Nome,Voto,Gf,Gs,Rp,Rs,Rf,Au,Amm,Esp,Ass,Asf,Gdv,Gdp,Giornata
0,4,P,SPORTIELLO,6.51,0,1,0,0,0,0,0,0,0,0,0,0,1
1,17,D,STENDARDO,6.03,0,0,0,0,0,0,0,0,0,0,0,0,1
2,15,D,MASIELLO A,5.51,0,0,0,0,0,0,0,0,0,0,0,0,1
3,10,D,CHERUBIN,6.52,0,0,0,0,0,0,0,0,0,0,0,0,1
4,13,D,DRAME',6.01,0,0,0,0,0,0,0,0,0,0,0,0,1


In [5]:
df_all_17 = merge_matchdays(2017)

In [6]:
df_all_17.head(5)

,Cod.,Ruolo,Nome,Voto,Gf,Gs,Rp,Rs,Rf,Au,Amm,Esp,Ass,Asf,Gdv,Gdp,Giornata
0,4,P,SPORTIELLO,4.53,0,4,0,0,0,0,0,0,0,0,0,0,1
1,16,D,RAIMONDI,5.04,0,0,0,0,0,0,1,0,0,0,0,0,1
2,11,D,CONTI,6.53,0,0,0,0,0,0,1,0,1,0,0,0,1
3,13,D,DRAME',5.04,0,0,0,0,0,0,0,0,0,0,0,0,1
4,695,D,TOLOI,4.53,0,0,0,0,0,0,0,0,0,0,0,0,1


In [7]:
df_all_18 = merge_matchdays(2018)

In [8]:
df_all_18.head(5)

,Cod.,Ruolo,Nome,Voto,Gf,Gs,Rp,Rs,Rf,Au,Amm,Esp,Ass,Asf,Gdv,Gdp,Giornata
0,316,P,BERISHA,6,0,1,0,0,0,0,0,0,0,0,0,0,1
1,2160,D,GOSENS,5.5,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2130,D,HATEBOER,6,0,0,0,0,0,0,0,0,0,0,0,0,1
3,15,D,MASIELLO A,5.5,0,0,0,0,0,0,0,0,0,0,0,0,1
4,2181,D,PALOMINO,6.5,0,0,0,0,0,0,0,0,0,0,0,0,1


In [9]:
def calc_fantavoto(df):
    df["Voto"] = df["Voto"].apply(float)
    df.loc[:, "Fantavoto"] = df["Voto"] + 3*df["Gf"] + 2*df["Rf"] - df["Rs"] + df["Ass"] + df["Asf"] - 0.5*df["Amm"] - df["Esp"] - df["Gs"] + 3*df["Rp"] + df["Gdv"]    
    return df

In [10]:
#Creo le liste per le varie statistiche e poi ce le aggiungo
def calc_stats(df):
    df = calc_fantavoto(df)
    #Divido il dataframe per nome
    Players = df.groupby(by = ["Nome"])
    nomi = []
    GP = []
    std = []
    mean = []
    for name, group in Players:
        nomi.append(name)
        #rimuovo i sei politici
        group = group[~group['Voto'].isin(['6*'])]
        GP.append(group.count()["Giornata"])
        std.append(group["Fantavoto"].std())
        mean.append(group["Fantavoto"].mean())
    df = pd.DataFrame({"Nome":nomi, "GP":GP,"STD":std,"Mean":mean})
    #rimuovo i non giocanti
    return df.dropna()

In [11]:
Final_16 = calc_stats(df_all_16)

In [12]:
Final_17 = calc_stats(df_all_17)

In [13]:
Final_18 = calc_stats(df_all_18)

In [14]:
def tit(df):
    df.loc[:,"Titolare"] = df["GP"]/38

In [15]:
tit(Final_16)
tit(Final_17)
tit(Final_18)

In [16]:
def merge_teams(df, year):
    df_listone = pd.read_excel(f"data/listoni/Statistiche_Fantacalcio_{year-1}-{year-2000}_Fantagazzetta.xlsx", skiprows=1)
    df_listone = df_listone[["Nome", "Squadra"]]
    return df.merge(df_listone, on="Nome")

In [17]:
Final_16 = merge_teams(Final_16, 2016)

In [18]:
Final_17 = merge_teams(Final_17, 2017)

In [19]:
Final_18 = merge_teams(Final_18, 2018)

In [20]:
Final_16.head()

,Nome,GP,STD,Mean,Titolare,Squadra
0,ABATE,26,0.946871,6.158462,0.684211,Milan
1,ACERBI,36,1.235275,6.664167,0.947368,Sassuolo
2,ACQUAFRESCA,4,0.294449,5.275000,0.105263,Bologna
3,ACQUAH,29,1.493965,6.145862,0.763158,Torino
4,ADNAN,28,0.851652,6.076786,0.736842,Udinese


**Merge titolarità 2019**

In [21]:
def get_name(x):
    if len(x.split()) == 1:
        return x
    else:
        x = [w for w in x.split() if not any(letter in w for letter in "abcdefghijklmnopqrstuvwxyz")]
        return " ".join(x)

In [22]:
df_ruoli = pd.read_excel(f"data/Quotazioni_Fantacalcio_Ruoli_Mantra.xlsx",header = 1)
df_tit = pd.read_excel(f"data/Quotazioni Ufficiali Fantacalcio Serie A 2018 2019.xlsx",header = 1)
df_tit["Nome"] = df_tit["Nome"].apply(get_name)


In [23]:
df_ruoli.head()

,Id,R,Nome,Squadra,Qt. A,Qt. I,Diff.
0,608,Pc,ZAPATA D,Atalanta,25,25,0
1,177,T;A,ILICIC,Atalanta,24,24,0
2,26,A,GOMEZ A,Atalanta,24,24,0
3,788,C,FREULER,Atalanta,16,16,0
4,15,Dd;Dc,MASIELLO A,Atalanta,15,15,0


In [24]:
df_tit = df_tit[["Nome", "Tit"]]

In [25]:
df_tit.head()

,Nome,Tit
NaN,CRISTIANO RONALDO,0.9
NaN,IMMOBILE,0.9
NaN,ICARDI,0.9
NaN,DYBALA,0.9
NaN,DZEKO,0.9


In [26]:
df = df_ruoli.merge(df_tit, how="left")

In [27]:
df.to_csv("data/final_19.csv")

In [28]:
Final_16.to_csv("data/final_16.csv")
Final_17.to_csv("data/final_17.csv")
Final_18.to_csv("data/final_18.csv")

**Find misspellings**

Ref: https://stackoverflow.com/a/6690850/5008494